In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime
import calendar
from scipy import interpolate
import os
import shutil
import csv

In [2]:
def pcoord(x, y): 
    """
    Convert x, y to polar coordinates r, az (geographic convention)
    r,az = pcoord(x, y)
    """
    r  = np.sqrt( x**2 + y**2 )
    az=np.degrees( np.arctan2(x, y) )
    # az[where(az<0.)[0]] += 360.
    az = (az+360.)%360.
    return r, az

def xycoord(r, az):
    """
    Convert r, az [degrees, geographic convention] to rectangular coordinates
    x,y = xycoord(r, az)
    """
    x = r * np.sin(np.radians(az))
    y = r * np.cos(np.radians(az))
    return x, y


In [3]:
fn = 'buoydata/micity2017.txt'

df = pd.read_csv(fn, skiprows=range(1,2), delim_whitespace = True, \
                    parse_dates={'date':[0,1,2,3,4]}, keep_date_col=False)

# Transfer data in "date" column to a column where it is stored as a datetime object
df['datetime'] = pd.to_datetime(df['date'], format = '%Y %m %d %H %M',utc=True)
df = df.drop(df.columns[[0,1,2,3,6,8,9,10,11,12, 13]], axis = 1)
df

,WVHT,DPD,MWD,datetime
0,0.01,11.64,999,2017-04-19 18:10:00+00:00
1,0.01,99.00,999,2017-04-19 18:20:00+00:00
2,0.01,12.15,45,2017-04-19 18:30:00+00:00
3,0.01,10.41,40,2017-04-19 18:40:00+00:00
4,0.01,6.58,227,2017-04-19 18:50:00+00:00
...,...,...,...,...
27893,0.70,2.29,66,2017-11-11 17:00:00+00:00
27894,0.92,2.29,999,2017-11-11 17:10:00+00:00
27895,0.91,2.29,999,2017-11-11 17:20:00+00:00
27896,0.72,2.29,61,2017-11-11 17:30:00+00:00


In [4]:
# calculate unix datetime
df['epoch']=(df['datetime'] - pd.Timestamp("1970-01-01",tz='utc')) // pd.Timedelta('1s')
df

,WVHT,DPD,MWD,datetime,epoch
0,0.01,11.64,999,2017-04-19 18:10:00+00:00,1492625400
1,0.01,99.00,999,2017-04-19 18:20:00+00:00,1492626000
2,0.01,12.15,45,2017-04-19 18:30:00+00:00,1492626600
3,0.01,10.41,40,2017-04-19 18:40:00+00:00,1492627200
4,0.01,6.58,227,2017-04-19 18:50:00+00:00,1492627800
...,...,...,...,...,...
27893,0.70,2.29,66,2017-11-11 17:00:00+00:00,1510419600
27894,0.92,2.29,999,2017-11-11 17:10:00+00:00,1510420200
27895,0.91,2.29,999,2017-11-11 17:20:00+00:00,1510420800
27896,0.72,2.29,61,2017-11-11 17:30:00+00:00,1510421400


In [5]:
# remove data with NaN values
df = df[df['WVHT'] < 99.0]
df = df[df['DPD'] <99.0]
df = df[df['MWD'] < 999]
df

,WVHT,DPD,MWD,datetime,epoch
2,0.01,12.15,45,2017-04-19 18:30:00+00:00,1492626600
3,0.01,10.41,40,2017-04-19 18:40:00+00:00,1492627200
4,0.01,6.58,227,2017-04-19 18:50:00+00:00,1492627800
5,0.01,7.47,36,2017-04-19 19:00:00+00:00,1492628400
6,0.01,7.63,269,2017-04-19 19:10:00+00:00,1492629000
...,...,...,...,...,...
27888,0.43,2.22,202,2017-11-11 16:10:00+00:00,1510416600
27891,0.48,2.29,190,2017-11-11 16:40:00+00:00,1510418400
27893,0.70,2.29,66,2017-11-11 17:00:00+00:00,1510419600
27896,0.72,2.29,61,2017-11-11 17:30:00+00:00,1510421400


In [11]:
# get only data from October 2017
octdf = df[pd.DatetimeIndex(df['datetime']).month == 10]
octdf

,WVHT,DPD,MWD,datetime,epoch
21970,0.74,4.00,2,2017-10-01 00:50:00+00:00,1506819000
21972,0.68,3.46,31,2017-10-01 01:10:00+00:00,1506820200
21974,0.74,3.33,42,2017-10-01 01:30:00+00:00,1506821400
21980,0.64,3.51,288,2017-10-01 02:30:00+00:00,1506825000
21982,0.57,3.67,38,2017-10-01 02:50:00+00:00,1506826200
...,...,...,...,...,...
26352,0.60,5.47,320,2017-10-31 22:40:00+00:00,1509489600
26353,0.58,5.20,243,2017-10-31 22:50:00+00:00,1509490200
26355,0.60,5.01,77,2017-10-31 23:10:00+00:00,1509491400
26357,0.60,5.23,298,2017-10-31 23:30:00+00:00,1509492600


In [7]:
# put data frame into a numpy array
waves = octdf[octdf.columns[[4,0,1,2]]].to_numpy()
print('array with shape',np.shape(waves))
print(waves[0])

array with shape (2787, 4)
[1.506819e+09 7.400000e-01 4.000000e+00 2.000000e+00]


In [8]:
#check first time in record
(pd.Timestamp("2017-10-01 00:50:00")-pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')

1506819000

In [9]:
# interpolators

#linear interpolator
linf = interpolate.interp1d(waves[:,0], waves[:,1:4], axis=0, kind='linear')
# Nearest Neighbor interpolator
nnf =interpolate.interp1d(waves[:,0], waves[:,0:4], axis=0, kind='nearest')

In [24]:
# set generic names
csvfile = "MCv2oct2017.csv"
owgtrainimgs = "D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/"+"owgtrainimgs"
directory = "D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017"

# create directory that will hold images with wave data
try:
    shutil.rmtree(owgtrainimgs)
    os.mkdir(owgtrainimgs)
except:
    os.mkdir(owgtrainimgs)
    print("couldn't find folder, make new one")

# delete the csv file
try:
    os.remove(csvfile)
except:
    print("couldn't find file, making new one")
    
counter = 0

for filename in os.listdir(directory):
    # Use string slicing to remove .jpg from filename
    size = len(filename)
    filename = filename[:size - 4]
    
    # get time from filename
    ti = calendar.timegm(datetime.datetime.strptime(filename, "%Y%m%d%H%M").timetuple())
    print (ti)
    
    try:
        # interpolate data
        zi = nnf(ti)
        wi = linf(ti)
        
        # add images that have wave data within 30 minutes (1800 seconds) of upload to the folder that will train the model
        timedif = abs(int(ti)-int(zi[0]))
        if timedif >= 1800:
            print("out of range {}".format(filename), timedif)
        else:
            with open(csvfile, "a") as text_file:
                text_file.write("{0:s}.jpg,{1:0.2f},{2:0.2f},{3:0.2f}\n".format(filename,wi[0],wi[1],zi[3]))
            source = "D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017"+"/"+filename+".jpg"
            destination = owgtrainimgs+"/"+filename+".jpg"
            print('moving {} from {} to {}'.format(filename, source, destination))
            shutil.copy(source, destination)
            
            # document that a file has been moved to the training folder
            counter = counter + 1
            
    
    except:
        print ('fail')

print (counter, "files moved")

1507149180
moving 201710042033 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710042033.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710042033.jpg
1509197580
moving 201710281333 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710281333.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710281333.jpg
1509042780
moving 201710261833 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710261833.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710261833.jpg
1507069980
moving 201710032233 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710032233.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710032233.jpg
1509039180
moving 201710261733 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710261733.jpg to D:/Webcam Images/MI City/2017_imgpre

moving 201710062033 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710062033.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710062033.jpg
1507919640
out of range 201710131834 5040
1508268840
moving 201710171934 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710171934.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710171934.jpg
1508344440
moving 201710181634 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710181634.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710181634.jpg
1508157240
moving 201710161234 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710161234.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710161234.jpg
1507916040
moving 201710131734 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710131734.jpg to D:/We

1509107580
moving 201710271233 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710271233.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710271233.jpg
1509046380
moving 201710261933 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710261933.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710261933.jpg
1509193980
moving 201710281233 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710281233.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710281233.jpg
1507415580
moving 201710072233 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710072233.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710072233.jpg
1507336440
moving 201710070034 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710070034.jpg to D:/Webcam Images/MI City/2017_imgpre

moving 201710052233 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710052233.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710052233.jpg
1508873580
moving 201710241933 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710241933.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710241933.jpg
1509204780
moving 201710281533 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710281533.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710281533.jpg
1508934780
moving 201710251233 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710251233.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710251233.jpg
1508592780
moving 201710211333 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710211333.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/vie

1508078040
moving 201710151434 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710151434.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710151434.jpg
1508175240
moving 201710161734 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710161734.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710161734.jpg
1507898040
out of range 201710131234 9960
1508506380
moving 201710201333 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710201333.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710201333.jpg
1508607180
moving 201710211733 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710211733.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710211733.jpg
1508682780
moving 201710221433 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710221433.j

1508520840
moving 201710201734 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710201734.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710201734.jpg
1507649580
moving 201710101533 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710101533.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710101533.jpg
1508160780
moving 201710161333 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710161333.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710161333.jpg
1508419980
moving 201710191333 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710191333.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710191333.jpg
1508099580
moving 201710152033 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710152033.jpg to D:/Webcam Images/MI City/2017_imgpre

moving 201710151534 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710151534.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710151534.jpg
1507901640
out of range 201710131334 6360
1508441640
moving 201710191934 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710191934.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710191934.jpg
1507829640
moving 201710121734 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710121734.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710121734.jpg
1509395640
moving 201710302034 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710302034.jpg to D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/owgtrainimgs/201710302034.jpg
1508430840
moving 201710191634 from D:/Webcam Images/MI City/2017_imgprep/view2/view2OWGimg/october2017/201710191634.jpg to D:/We